In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 1min 35s, sys: 1.21 s, total: 1min 36s
Wall time: 1min 37s


In [6]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
condition_1 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_mondo.df['object_id'].str.contains("MONDO")
condition_4 = msdf_mondo.df['subject_id'].str.contains("ICD10CM")
msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [7]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_lex.df['object_id'].str.contains("MONDO")
condition_4 = msdf_lex.df['subject_id'].str.contains("ICD10CM")
msdf_lex.df = msdf_lex.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_lex.df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
128740,ICD10CM:A00,Cholera,skos:exactMatch,MONDO:0015766,cholera,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,cholera
128746,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,typhoid fever
128747,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,typhoid fever
128751,ICD10CM:A01.1,Paratyphoid fever A,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,NaN,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever a
128755,ICD10CM:A01.2,Paratyphoid fever B,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,NaN,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever b


In [8]:
duplicate_df_lex = msdf_lex.df.copy()
duplicate_df_mondo = msdf_mondo.df.head().copy()

duplicate_df_lex = duplicate_df_lex.rename(columns={"subject_id": "object_id","object_id": "subject_id" })
duplicate_df_mondo = duplicate_df_mondo.rename(columns={"subject_id": "object_id","object_id": "subject_id" })

desired_col_sequnce = ["subject_id","subject_label", "predicate_id", "object_id", "object_label", "mapping_justification"]
duplicate_df_lex = duplicate_df_lex[desired_col_sequnce]
duplicate_df_mondo = duplicate_df_mondo[desired_col_sequnce]


msdf_lex.df = pd.concat([duplicate_df_lex, msdf_lex.df]).drop_duplicates()
msdf_mondo.df = pd.concat([duplicate_df_mondo, msdf_mondo.df]).drop_duplicates()

msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification
51,ICD10CM:E30.1,precocious puberty,skos:exactMatch,MONDO:0000088,Precocious puberty,Unspecified
111,ICD10CM:Q20.3,transposition of the great arteries,skos:broadMatch,MONDO:0000153,Discordant ventriculoarterial connection,Unspecified
113,ICD10CM:Q20.5,transposition of the great arteries,skos:broadMatch,MONDO:0000153,Discordant atrioventricular connection,Unspecified
202,ICD10CM:E83,mineral metabolism disease,skos:exactMatch,MONDO:0000226,Disorders of mineral metabolism,Unspecified
222,ICD10CM:A26,erysipeloid,skos:exactMatch,MONDO:0000237,Erysipeloid,Unspecified


In [9]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
14662,MONDO:0005100,systemic sclerosis,skos:broadMatch,ICD10CM:M34.8,Other forms of systemic sclerosis,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50418,MONDO:0015153,obsolete autosomal monosomy,skos:broadMatch,ICD10CM:Q93.2,"Chromosome replaced with ring, dicentric or is...",Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62812,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q71.6,Lobster-claw hand,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53421,MONDO:0016271,adenoid cystic carcinoma of the corpus uteri,skos:broadMatch,ICD10CM:C54.0,Malignant neoplasm of isthmus uteri,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
57938,MONDO:0018076,tuberculosis,skos:exactMatch,ICD10CM:A15-A19,Tuberculosis (A15-A19),Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [10]:
comparison_df[comparison_df['object_id'] == "ICD10CM:D35.2"]

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
65999,MONDO:0021439,benign neoplasm of pituitary gland,skos:exactMatch,ICD10CM:D35.2,Benign neoplasm of pituitary gland,Unspecified,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN
130884,MONDO:0021439,Benign neoplasm of pituitary gland,skos:exactMatch,ICD10CM:D35.2,benign neoplasm of pituitary gland,semapv:LexicalMatching,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN


In [11]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
14662,MONDO:0005100,systemic sclerosis,skos:broadMatch,ICD10CM:M34.8,Other forms of systemic sclerosis,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50418,MONDO:0015153,obsolete autosomal monosomy,skos:broadMatch,ICD10CM:Q93.2,"Chromosome replaced with ring, dicentric or is...",Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62812,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q71.6,Lobster-claw hand,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53421,MONDO:0016271,adenoid cystic carcinoma of the corpus uteri,skos:broadMatch,ICD10CM:C54.0,Malignant neoplasm of isthmus uteri,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
57938,MONDO:0018076,tuberculosis,skos:exactMatch,ICD10CM:A15-A19,Tuberculosis (A15-A19),Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [12]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
131151,ICD10CM:D73.1,Hypersplenism,skos:exactMatch,MONDO:0006795,hypersplenism,semapv:LexicalMatching,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,hypersplenism
131154,ICD10CM:D73.1,Hypersplenism,skos:exactMatch,MONDO:0006795,hypersplenism,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,hypersplenism
138155,ICD10CM:P39.0,Neonatal infective mastitis,skos:exactMatch,MONDO:0001736,neonatal infective mastitis,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,neonatal infective mastitis
129725,ICD10CM:B36.0,Pityriasis versicolor,skos:exactMatch,MONDO:0005915,pityriasis versicolor,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,pityriasis versicolor
131708,ICD10CM:E70.331,Hermansky-Pudlak syndrome,skos:exactMatch,MONDO:0019312,Hermansky-Pudlak syndrome,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,hermansky-pudlak syndrome


In [13]:
# Inspect why these are missing from SSSOM mappings

unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))
# Add distances
# Add Levenshtein distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# Add Jaccard Index [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()


/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_39459/653029316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,levenshtein_dist,jaccard_index,monge_elkan
57938,MONDO:0018076,tuberculosis,skos:exactMatch,ICD10CM:A15-A19,Tuberculosis (A15-A19),Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,10,0.454545,1.916667
53120,MONDO:0016158,narcolepsy-cataplexy syndrome,skos:exactMatch,ICD10CM:G47.4,Narcolepsy and cataplexy,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,14,0.290323,1.967895
2760,MONDO:0001291,brain compression,skos:exactMatch,ICD10CM:G93.5,Compression of brain,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,15,0.150000,1.941176
67773,MONDO:0024644,myocardial ischemia,skos:exactMatch,ICD10CM:I20-I25,Ischemic heart diseases (I20-I25),Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,25,0.631579,1.955679
71799,MONDO:0100280,Waldenstrom macroglobulinemia,skos:exactMatch,ICD10CM:C88.0,WaldenstrÃ¶m macroglobulinemia,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,2,0.096774,1.965517


In [14]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
14662,MONDO:0005100,systemic sclerosis,skos:broadMatch,ICD10CM:M34.8,Other forms of systemic sclerosis,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
50418,MONDO:0015153,obsolete autosomal monosomy,skos:broadMatch,ICD10CM:Q93.2,"Chromosome replaced with ring, dicentric or is...",Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62812,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q71.6,Lobster-claw hand,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53421,MONDO:0016271,adenoid cystic carcinoma of the corpus uteri,skos:broadMatch,ICD10CM:C54.0,Malignant neoplasm of isthmus uteri,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
57938,MONDO:0018076,tuberculosis,skos:exactMatch,ICD10CM:A15-A19,Tuberculosis (A15-A19),Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [15]:
%%time
df_dict = split_dataframe(combined_msdf)

/opt/anaconda3/envs/mondo-ingest/lib/python3.9/site-packages/sssom/parsers.py:864: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  subject_prefixes = set(msdf.df[SUBJECT_ID].str.split(":", 1, expand=True)[0])
/opt/anaconda3/envs/mondo-ingest/lib/python3.9/site-packages/sssom/parsers.py:865: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  object_prefixes = set(msdf.df[OBJECT_ID].str.split(":", 1, expand=True)[0])


CPU times: user 5.96 s, sys: 40.9 ms, total: 6 s
Wall time: 11.8 s


In [16]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

['mondo_exactmatch_icd10cm',
 'mondo_narrowmatch_icd10cm',
 'mondo_closematch_icd10cm',
 'mondo_broadmatch_icd10cm',
 'icd10cm_exactmatch_mondo',
 'icd10cm_narrowmatch_mondo',
 'icd10cm_closematch_mondo',
 'icd10cm_broadmatch_mondo']

In [17]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [18]:
df_dict['mondo_exactmatch_icd10cm'].df

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment
0,MONDO:0000022,Nocturnal enuresis,skos:exactMatch,ICD10CM:N39.44,nocturnal enuresis,semapv:LexicalMatching,LEXMATCH
1,MONDO:0000078,Congenital malformation syndromes predominantl...,skos:exactMatch,ICD10CM:Q87.0,acrocephalopolysyndactyly,semapv:LexicalMatching,LEXMATCH
2,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Precocious puberty,Unspecified,COMMON_TO_BOTH
3,MONDO:0000088,Precocious puberty,skos:exactMatch,ICD10CM:E30.1,precocious puberty,semapv:LexicalMatching,COMMON_TO_BOTH
4,MONDO:0000156,Craniosynostosis,skos:exactMatch,ICD10CM:Q75.0,trigonocephaly,semapv:LexicalMatching,LEXMATCH
...,...,...,...,...,...,...,...
3562,MONDO:0800026,Congenital central alveolar hypoventilation sy...,skos:exactMatch,ICD10CM:G47.35,"central hypoventilation syndrome, congenital, ...",semapv:LexicalMatching,LEXMATCH
3563,MONDO:0800029,Idiopathic pulmonary fibrosis,skos:exactMatch,ICD10CM:J84.112,interstitial lung disease 2,semapv:LexicalMatching,COMMON_TO_BOTH
3564,MONDO:0800029,interstitial lung disease 2,skos:exactMatch,ICD10CM:J84.112,Idiopathic pulmonary fibrosis,Unspecified,COMMON_TO_BOTH
3565,MONDO:8000010,antiphospholipid syndrome,skos:exactMatch,ICD10CM:D68.61,Antiphospholipid syndrome,Unspecified,COMMON_TO_BOTH
